<a href="https://colab.research.google.com/github/ZicoDiegoRR/stable_diffusion_xl_colab_ui/blob/main/beta_stable_diffusion_xl_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###<font color="black"> » <b><font color="red">Installing Dependencies </b>💿</font> <font color="black"> «
#####ㅤRun this cell first before creating images!

In [ ]:
#@markdown <b>Run this first to install essential libraries!</b><br>
#@markdown <small><p>Required to use the generator.
from IPython.display import clear_output
print("⚙️ | Downloading libraries...")
!pip install diffusers
!pip install torch torchvision torchaudio
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install accelerate
!pip install opencv-python
!pip install peft
!pip install --upgrade huggingface_hub
!pip install compel
!pip install controlnet-aux
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
clear_output()
print("📁 | All essential libraries have been downloaded.")
print("🖌 | You can start generating images now.")

###<font color="black"> »<b><font color="orange">Running Stable Diffusion</b> 🔧</font> <font color="black"> «

In [ ]:
from PIL import Image as ImagePIL
from compel import Compel, ReturnedEmbeddingsType
from controlnet_aux import OpenposeDetector
from diffusers import ControlNetModel, StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, StableDiffusionXLControlNetPipeline, AutoPipelineForInpainting, AutoencoderKL
from diffusers import DDPMScheduler, DPMSolverMultistepScheduler, DPMSolverSinglestepScheduler, KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, HeunDiscreteScheduler, LMSDiscreteScheduler, DEISMultistepScheduler, UniPCMultistepScheduler, DDIMScheduler, PNDMScheduler
from diffusers.utils import load_image, make_image_grid
from huggingface_hub import login
from transformers import pipeline as pipe
from transformers import CLIPVisionModelWithProjection
from google.colab import drive
from IPython.display import display, clear_output
from safetensors.torch import load_file
from io import BytesIO
import ipywidgets as widgets
import numpy as np
import time
import cv2
import re
import os
import subprocess
import os.path
import torch
import random
import json
import math

#@markdown <b>Run the cell to start!</b>

#@markdown <small>Just run the cell and enjoy. (required to run the cell above first)</small>

#@markdown <small>You can disable Google Drive by not permitting the notebook to access your Google Drive storage.</small>

#@markdown <small>If the runtime got restarted, just run it again.</small>

# Function to load parameters config
def load_param(filename):
    try:
        with open(filename, 'r') as f:
            params = json.load(f)
        return params
    except FileNotFoundError:
        return []

# Function to save the data to a json
def save_last(filename, data, type):
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_data = json.load(file)
        else:
            existing_data = {}

        if type == "[Text-to-Image]":
            existing_data['text2img'] = data
        elif type == "[ControlNet]":
            existing_data['controlnet'] = data
        elif type == "[Inpainting]":
            existing_data['inpaint'] = data
        with open(filename, 'w') as file:
            json.dump(existing_data, file, indent=4)
    except Exception as e:
        print(f"Error occurred: {e}")

# Function to load last-generated image
def load_last(filename, type):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data.get(type, None)
    except (FileNotFoundError, json.JSONDecodeError):
        return None

# Function to load the saved data from a json
def load_number(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data['saved']
    except (FileNotFoundError, KeyError):
        return None

# Function to save the data to a json
def save_number(filename, data):
    with open(filename, 'w') as file:
        json.dump({'saved': data}, file)

#Function to save parameters config (had to make separate JSON def to avoid confusion)
def save_param(path, data):
    with open(path, 'w') as file:
        json.dump(data, file)

# Function to convert image into depth map
def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

# Only for display in output, nothing crazy
def get_depth_map_display(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    return image

# Function to restart the runtime to free up some of the VRAM if there's a change in model or the pipeline
def restart(new, old):
    print(f"New model is found. Your previous one ({old}) is different than your new one ({new}).")
    print("Restarting the runtime is necessary to load the new one.")
    time.sleep(2)
    print("Restarting the runtime...")
    time.sleep(0.5)
    os.kill(os.getpid(), 9)

def param_default(): # For parameters validation and/or parameters reset
  default_cfg = [
      "",
      "",
      "Safe Tensor (.safetensors)",
      "",
      1024,
      1024,
      "Default (defaulting to the model)",
      12,
      6,
      "",
      2,
      "",
      "",
      100,
      240,
      "",
      False,
      0.7,
      "",
      False,
      0.7,
      "",
      False,
      0.7,
      "pre-generated text2image image",
      "",
      False,
      0.9,
      "None",
      "",
      0.7,
      False,
      False,
      False,
      False,
      False,
      "",
      "",
      0.3,
      "",
      ""
  ]
  return default_cfg

def param_constructor(): # For generating parameters based on the widgets to be saved
  param = [
        prompt_widget.value,
        model_widget.value,
        model_format_widget.value,
        negative_prompt_widget.value,
        width_slider.value,
        height_slider.value,
        scheduler_dropdown.value,
        steps_slider.value,
        scale_slider.value,
        vae_link_widget.value,
        clip_skip_slider.value,
        lora_urls_widget.value,
        weight_scale_widget.value,
        canny_min_slider.value,
        canny_max_slider.value,
        canny_link_widget.value,
        canny_toggle.value,
        canny_strength_slider.value,
        depth_map_link_widget.value,
        depth_map_toggle.value,
        depth_strength_slider.value,
        openpose_link_widget.value,
        openpose_toggle.value,
        openpose_strength_slider.value,
        inpainting_image_dropdown.value,
        mask_image_widget.value,
        inpainting_toggle.value,
        inpainting_strength_slider.value,
        ip_adapter_dropdown.value,
        ip_image_link_widget.value,
        ip_adapter_strength_slider.value,
        freeze_widget.value,
        karras_bool.value,
        vpred_bool.value,
        sgmuniform_bool.value,
        res_betas_zero_snr.value,
        vae_config.value,
        reference_image_link_widget.value,
        denoising_strength_slider.value,
        ti_urls_widget.value,
        ti_tokens_widget.value
    ] # 41 values in total
  return param

def all_widgets(): # Packing all widgets into a single VBox
  all_usable_widgets = widgets.VBox([
        prompt_widget,
        model_widget,
        model_format_widget,
        negative_prompt_widget,
        width_slider,
        height_slider,
        scheduler_dropdown,
        steps_slider,
        scale_slider,
        vae_link_widget,
        clip_skip_slider,
        lora_urls_widget,
        weight_scale_widget,
        canny_min_slider,
        canny_max_slider,
        canny_link_widget,
        canny_toggle,
        canny_strength_slider,
        depth_map_link_widget,
        depth_map_toggle,
        depth_strength_slider,
        openpose_link_widget,
        openpose_toggle,
        openpose_strength_slider,
        inpainting_image_dropdown,
        mask_image_widget,
        inpainting_toggle,
        inpainting_strength_slider,
        ip_adapter_dropdown,
        ip_image_link_widget,
        ip_adapter_strength_slider,
        freeze_widget,
        karras_bool,
        vpred_bool,
        sgmuniform_bool,
        res_betas_zero_snr,
        vae_config,
        reference_image_link_widget,
        denoising_strength_slider,
        ti_urls_widget,
        ti_tokens_widget
    ])
  return all_usable_widgets

# Loading the saved config for the IPyWidgets
try:
    drive.mount('/content/gdrive', force_remount=True)
except Exception as e:
    print("Excluding Google Drive storage...")
    time.sleep(1.5)

if os.path.exists("/content/gdrive/MyDrive"):
    base_path = "/content/gdrive/MyDrive"
    Save_and_Connect_To_GDrive = True
else:
    base_path = "/content"
    Save_and_Connect_To_GDrive = False
if os.path.exists(f"{base_path}/parameters.json"):
  cfg = load_param(os.path.join(f"{base_path}", "parameters.json"))
  print(f"Found a config at {base_path}/parameters.json.")
  if not os.path.exists(f"{base_path}/Saved Parameters"):
    os.mkdir(f"{base_path}/Saved Parameters")
  save_param(os.path.join(f"{base_path}/Saved Parameters/", "main_parameters.json"), cfg)
elif not os.path.exists(f"{base_path}/parameters.json") or os.path.exists(f"{base_path}/Saved Parameters/main_parameters.json"):
  cfg = load_param(os.path.join(f"{base_path}/Saved Parameters/", "main_parameters.json"))
  print(f"Found a config at {base_path}/Saved Parameters/main_parameters.json.")
if cfg:
  if len(cfg) < 41:
    for i in range(41 - len(cfg)):
      cfg.append(None)
else:
    print("No saved config found. Defaulting...")
    time.sleep(1)

# Validating the loaded parameters (compatibility feature for using parameters from previous versions)
if cfg:
  default_cfg = param_default()
  for i in range(len(cfg)):
    if cfg[i] is None:
      cfg[i] = default_cfg[i]
  save_param(os.path.join(f"{base_path}/Saved Parameters/", "main_parameters.json"), cfg)

# IPyWidgets⬇️
# ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

# Image Generation Section
prompt_widget = widgets.Textarea(value=cfg[0] if cfg else "", placeholder="Enter your prompt here")
negative_prompt_widget = widgets.Textarea(value=cfg[3] if cfg else "", placeholder="What you don't want to see?")
prompts_section = widgets.HBox([widgets.VBox([widgets.Label(value="Prompt:"), prompt_widget]), widgets.VBox([widgets.Label(value="Negative prompt:"), negative_prompt_widget])])

prompt_widget.layout.flex = "1"
negative_prompt_widget.layout.flex = "1"

model_widget = widgets.Text(value=cfg[1] if cfg else "", placeholder="HF's repository or direct URL")
model_format_widget = widgets.Dropdown(options=["Pickle Tensor (.ckpt)", "Safe Tensor (.safetensors)"], value=cfg[2] if cfg else "Safe Tensor (.safetensors)", description="Model Format and Link",)
model_input_section = widgets.HBox([model_format_widget, model_widget])

def reference_image_upload_handler(change):
  if not os.path.exists("/content/img2img/"):
    os.mkdir("/content/img2img/")
  for file_info in reference_image_upload_widget.value.items():
    ref_uploaded_image = file_info[1]["content"]
    with open("/content/img2img/temp.png", "wb") as up:
      up.write(ref_uploaded_image)
  reference_image_link_widget.value = "/content/img2img/temp.png"

reference_image_link_widget = widgets.Text(placeholder="Img2Img reference link", description="Reference Image", value=cfg[37] if cfg and (not cfg[37].startswith("/content/img2img/") or os.path.exists("/content/img2img/")) else "")
reference_image_upload_widget = widgets.FileUpload(accept="image/*", multiple=False)
denoising_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.01, description="Denoising Strength", value=cfg[38] if cfg else 0.3)
reference_image_section = widgets.VBox([widgets.HBox([reference_image_link_widget, reference_image_upload_widget]), widgets.HBox([denoising_strength_slider, widgets.HTML(value="Low value means similar to the original image.")])])
reference_image_upload_widget.observe(reference_image_upload_handler, names="value")

width_slider = widgets.IntSlider(min=512, max=1536, step=64, value=cfg[4] if cfg else 1024, description="Width")
height_slider = widgets.IntSlider(min=512, max=1536, step=64, value=cfg[5] if cfg else 1024, description="Height")
image_resolution_section = widgets.HBox([width_slider, height_slider])

steps_slider = widgets.IntText(value=cfg[7] if cfg else 12, description="Steps")
scale_slider = widgets.FloatSlider(min=1, max=12, step=0.1, value=cfg[8] if cfg else 6, description="Scale")
clip_skip_slider = widgets.IntSlider(min=0, max=12, step=1, value=cfg[10] if cfg else 2, description="Clip Skip")
generation_parameter_section = widgets.VBox([steps_slider, widgets.HBox([scale_slider, clip_skip_slider])])

def scheduler_dropdown_handler(change): # Function to show or hide scheduler booleans
  if change["new"] != "Default (defaulting to the model)":
    scheduler_settings.children = [scheduler_dropdown, karras_bool, vpred_bool, sgmuniform_bool, res_betas_zero_snr, widgets.HTML(value="Rescaling the betas to have zero terminal SNR helps to achieve vibrant color, but not necessary.")]
  else:
    scheduler_settings.children = [scheduler_dropdown]

scheduler_dropdown = widgets.Dropdown(
    options=[
        "Default (defaulting to the model)", "DPM++ 2M", "DPM++ 2M SDE",
        "DPM++ SDE", "DPM2", "DDPM",
        "DPM2 a", "DDIM", "PNDM", "Euler", "Euler a", "Heun", "LMS",
        "DEIS", "UniPC"
    ],
    value=cfg[6] if cfg else "Default (defaulting to the model)",
    description="Scheduler",
)
karras_bool = widgets.Checkbox(value=cfg[32] if cfg else False, description="Enable Karras")
vpred_bool = widgets.Checkbox(value=cfg[33] if cfg else False, description="Enable V-prediction")
sgmuniform_bool = widgets.Checkbox(value=cfg[34] if cfg else False, description="Enable SGMUniform")
res_betas_zero_snr = widgets.Checkbox(value=cfg[35] if cfg else False, description="Rescale beta zero SNR")
scheduler_settings = widgets.VBox([scheduler_dropdown])

scheduler_dropdown.observe(scheduler_dropdown_handler, names="value")
scheduler_dropdown_handler({"new": scheduler_dropdown.value})

vae_link_widget = widgets.Text(value=cfg[9] if cfg else "", description="VAE", placeholder="VAE model link")
vae_config = widgets.Text(value=cfg[36] if cfg else "", placeholder="VAE config link")
vae_section = widgets.HBox([vae_link_widget, vae_config])

token_widget = widgets.Text(description="CivitAI Token", placeholder="Avoid 401 error from CivitAI")
freeze_widget = widgets.Checkbox(description="Use the same seed", value=cfg[31] if cfg else False)

img2img_settings = widgets.VBox([
    prompts_section,
    model_input_section,
    image_resolution_section,
    generation_parameter_section,
    reference_image_section,
    scheduler_settings,
    vae_section,
    freeze_widget,
    token_widget,
    widgets.HTML(value="For safety reason, your token <b>won't be saved</b>.")
])

text2img_settings = widgets.VBox([
    prompts_section,
    model_input_section,
    image_resolution_section,
    generation_parameter_section,
    scheduler_settings,
    vae_section,
    freeze_widget,
    token_widget,
    widgets.HTML(value="For safety reason, your token <b>won't be saved</b>.")
])

# LoRA Section
def lora_click(link, scale): # Function to add widgets after clicking the plus button
  lora_url_input = widgets.Text(value=link, placeholder="Input the link here", description="Direct URL")
  lora_scale_input = widgets.FloatSlider(value=scale, min=-5, max=5, step=0.1, description="Weight Scale")
  lora_remove_button = widgets.Button(description="X", button_style='danger', layout=widgets.Layout(width='30px', height='30px'))

  lora_nested_vbox.children += (lora_url_input, lora_scale_input, lora_remove_button,)
  lora_remove_button.on_click(lambda b: lora_remover(list(lora_nested_vbox.children).index(lora_remove_button) - 2, list(lora_nested_vbox.children).index(lora_remove_button) - 1, list(lora_nested_vbox.children).index(lora_remove_button)))
  lora_settings.children = [lora_add, lora_nested_vbox]

def lora_reader(): # Function to process every value from the widgets into two strings to be fed into the main logic (I'm too lazy to change the main code)
  collected_lora_urls = ""
  collected_lora_scales = ""
  for i in range(len(lora_nested_vbox.children)):
    if i % 3 == 0:
      if lora_nested_vbox.children[i].value != "":
        collected_lora_urls += (lora_nested_vbox.children[i].value + ",")
    elif i % 3 == 1:
      if lora_nested_vbox.children[i - 1].value != "":
        collected_lora_scales += (str(lora_nested_vbox.children[i].value) + ",")
  return collected_lora_urls, collected_lora_scales

def lora_remover(link, scale, remove_button): # Function to remove lora (only the widgets, not the actual file)
  lora_nested_list = list(lora_nested_vbox.children)
  lora_nested_list.pop(remove_button)
  lora_nested_list.pop(scale)
  lora_nested_list.pop(link)
  lora_nested_vbox.children = tuple(lora_nested_list)

def lora_reader_upon_starting(): # Function to add widgets based on pre-existing URLs from the saved parameter
  lora_links = [word for word in re.split(r"\s*,\s*", lora_urls_widget.value) if word]
  lora_scales = [float(word) for word in re.split(r"\s*,\s*", weight_scale_widget.value) if word]
  for i in range(len(lora_links)):
    lora_click(lora_links[i], lora_scales[i])

lora_urls_widget = widgets.Text(value=cfg[11] if cfg else "")
weight_scale_widget = widgets.Text(value=cfg[12] if cfg else "")

lora_add = widgets.Button(description="+", button_style='success', layout=widgets.Layout(width='30px', height='30px'))
lora_nested_vbox = widgets.VBox()
lora_settings = widgets.VBox([lora_add])

lora_add.on_click(lambda b: lora_click("", 1.0))

# Textual Inversion or Embeddings Section
def ti_click(link, token):  # Function to add widgets after clicking the plus button
    ti_url_input = widgets.Text(value=link, placeholder="Input the link here", description="Direct URL")
    ti_tokens_input = widgets.Text(value=token, placeholder="Activation tag", description="Token")
    ti_remove_button = widgets.Button(description="X", button_style='danger', layout=widgets.Layout(width='30px', height='30px'))

    ti_nested_vbox.children += (ti_url_input, ti_tokens_input, ti_remove_button,)
    ti_remove_button.on_click(lambda b: ti_remover(
        list(ti_nested_vbox.children).index(ti_remove_button) - 2,
        list(ti_nested_vbox.children).index(ti_remove_button) - 1,
        list(ti_nested_vbox.children).index(ti_remove_button)
    ))
    ti_settings.children = [ti_add, ti_nested_vbox]

def ti_reader():  # Function to process every value into two strings to be fed into the main logic
    collected_ti_urls = ""
    collected_ti_tokens = ""
    for i in range(len(ti_nested_vbox.children)):
        if i % 3 == 0:
          if ti_nested_vbox.children[i].value != "":
            collected_ti_urls += (ti_nested_vbox.children[i].value + ",")
        elif i % 3 == 1:
          if ti_nested_vbox.children[i - 1].value != "":
            collected_ti_tokens += (ti_nested_vbox.children[i].value + ",")
    return collected_ti_urls, collected_ti_tokens

def ti_remover(link, token, remove_button):  # Function to remove textual inversion widgets
    ti_nested_list = list(ti_nested_vbox.children)
    ti_nested_list.pop(remove_button)
    ti_nested_list.pop(token)
    ti_nested_list.pop(link)
    ti_nested_vbox.children = tuple(ti_nested_list)

def ti_reader_upon_starting():  # Function to add widgets from saved parameters
    ti_links = [word for word in re.split(r"\s*,\s*", ti_urls_widget.value)]
    if not ti_links[-1]:
      ti_links.pop(-1)
    ti_tokens = [word for word in re.split(r"\s*,\s*", ti_tokens_widget.value)]
    if not ti_tokens[-1]:
      ti_tokens.pop(-1)
    if len(ti_tokens) < len(ti_links):
      for i in range(len(ti_links) - len(ti_tokens)):
        ti_tokens.append("")
    for i in range(len(ti_links)):
        ti_click(ti_links[i], ti_tokens[i])

ti_urls_widget = widgets.Text(value=cfg[39] if cfg else "")
ti_tokens_widget = widgets.Text(value=cfg[40] if cfg else "")

ti_add = widgets.Button(description="+", button_style='success', layout=widgets.Layout(width='30px', height='30px'))
ti_nested_vbox = widgets.VBox()
ti_settings = widgets.VBox([ti_add])
ti_tab = widgets.VBox([widgets.HTML(value="Due to the architecture, you must pass the activation tag in the Token widget. Leaving it blank will skip the embeddings from being loaded."), ti_settings])

ti_add.on_click(lambda b: ti_click("", ""))

# ControlNet Section
controlnet_dropdown_choice = ["Link", "Upload", "Last Generated Text2Img", "Last Generated ControlNet", "Last Generated Inpainting"]
def controlnet_dropdown_handler(type, value): # Function to change the image reference based on the selected option in the dropdown
  controlnet_url_widgets_list = [canny_link_widget, depth_map_link_widget, openpose_link_widget]
  controlnet_upload_widgets_list = [canny_upload, depth_upload, openpose_upload]

  controlnet_type = 0 if type == "canny" else 1 if type == "depth" else 2
  controlnet_children = list(canny_settings.children) if controlnet_type == 0 else list(depth_settings.children) if controlnet_type == 1 else list(openpose_settings.children)
  if value == "Link":
    if controlnet_upload_widgets_list[controlnet_type] in controlnet_children:
      controlnet_children.pop(2)
    controlnet_children.insert(2, controlnet_url_widgets_list[controlnet_type])
  elif value == "Upload":
    if controlnet_url_widgets_list[controlnet_type] in controlnet_children:
      controlnet_children.pop(2)
    controlnet_children.insert(2, controlnet_upload_widgets_list[controlnet_type])
  else:
    if controlnet_url_widgets_list[controlnet_type] in controlnet_children:
      controlnet_children.pop(2)
    for i in range(len(controlnet_url_widgets_list)):
      if controlnet_url_widgets_list[i] in controlnet_children:
        controlnet_children.remove(controlnet_url_widgets_list[i])
      if controlnet_upload_widgets_list[i] in controlnet_children:
        controlnet_children.remove(controlnet_upload_widgets_list[i])
    if controlnet_type == 0:
      canny_link_widget.value = "" if value == "Last Generated Text2Img" else "controlnet" if value == "Last Generated ControlNet" else "inpaint"
    elif controlnet_type == 1:
      depth_map_link_widget.value = "" if value == "Last Generated Text2Img" else "controlnet" if value == "Last Generated ControlNet" else "inpaint"
    else:
      openpose_link_widget.value = "" if value == "Last Generated Text2Img" else "controlnet" if value == "Last Generated ControlNet" else "inpaint"

  if controlnet_type == 0:
    canny_settings.children = tuple(controlnet_children)
  elif controlnet_type == 1:
    depth_settings.children = tuple(controlnet_children)
  else:
    openpose_settings.children = tuple(controlnet_children)

def canny_popup(change): # Function to display canny settings if true
  if change["new"]:
    canny_settings.children = [canny_toggle, canny_dropdown, canny_min_slider, canny_max_slider, canny_strength_slider]
  else:
    canny_settings.children = [canny_toggle]

def canny_dropdown_handler(change): # Function to attach the canny dropdown to the controlnet dropdown handler
  controlnet_dropdown_handler("canny", change["new"])

def canny_upload_handler(change): # Function to load the path of the uploaded image to the image link
  if not os.path.exists("/content/canny/"):
    os.mkdir("/content/canny/")
  for file_info in canny_upload.value.items():
    canny_uploaded_image = file_info[1]["content"]
    with open("/content/canny/temp.png", "wb") as up:
      up.write(canny_uploaded_image)
  canny_link_widget.value = "/content/canny/temp.png"

canny_upload = widgets.FileUpload(accept="image/*", multiple=False)
canny_link_widget = widgets.Text(value=cfg[15] if cfg and (not cfg[15].startswith("/content/canny/") or os.path.exists("/content/canny/")) else "", description="Canny Link", placeholder="Image link")

canny_dropdown = widgets.Dropdown(options=controlnet_dropdown_choice, value=controlnet_dropdown_choice[2] if canny_link_widget.value == "" else controlnet_dropdown_choice[3] if canny_link_widget.value == "controlnet" else controlnet_dropdown_choice[4] if canny_link_widget.value == "inpaint" else controlnet_dropdown_choice[0], disabled=False, description="Reference Image")
canny_min_slider = widgets.IntSlider(min=10, max=500, step=5, value=cfg[13] if cfg else 100, description="Min Threshold")
canny_max_slider = widgets.IntSlider(min=100, max=750, step=5, value=cfg[14] if cfg else 240, description="Max Threshold")
canny_toggle = widgets.Checkbox(value=cfg[16] if cfg else False, description="Enable Canny")
canny_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[17] if cfg else 0.7, description="Canny Strength")
canny_settings = widgets.VBox([canny_toggle])

canny_popup({"new": canny_toggle.value})
canny_upload.observe(canny_upload_handler, names="value")

def depthmap_popup(change): # Function to display depth map settings if true
  if change["new"]:
    depth_settings.children = [depth_map_toggle, depthmap_dropdown, depth_strength_slider]
  else:
    depth_settings.children = [depth_map_toggle]

def depthmap_dropdown_handler(change): # Function to attach the canny dropdown to the controlnet dropdown handler
  controlnet_dropdown_handler("depth", change["new"])

def depthmap_upload_handler(change): # Function to load the path of the uploaded image to the image link
  if not os.path.exists("/content/depthmap/"):
    os.mkdir("/content/depthmap/")
  for file_info in depth_upload.value.items():
    depth_uploaded_image = file_info[1]["content"]
    with open("/content/depthmap/temp.png", "wb") as up:
      up.write(depth_uploaded_image)
  depth_map_link_widget.value = "/content/depthmap/temp.png"

depth_upload = widgets.FileUpload(accept="image/*", multiple=False)
depth_map_link_widget = widgets.Text(value=cfg[18] if cfg and (not cfg[18].startswith("/content/depthmap/") or os.path.exists("/content/depthmap/")) else "", description="DepthMap Link", placeholder="Image link")

depthmap_dropdown = widgets.Dropdown(options=controlnet_dropdown_choice, value=controlnet_dropdown_choice[2] if depth_map_link_widget.value == "" else controlnet_dropdown_choice[3] if depth_map_link_widget.value == "controlnet" else controlnet_dropdown_choice[4] if depth_map_link_widget.value == "inpaint" else controlnet_dropdown_choice[0], disabled=False, description="Reference Image")
depth_map_toggle = widgets.Checkbox(value=cfg[19] if cfg else False, description="Enable Depth Map")
depth_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[20] if cfg else 0.7, description="Depth Strength")
depth_settings = widgets.VBox([depth_map_toggle])

depthmap_popup({"new": depth_map_toggle.value})
depth_upload.observe(depthmap_upload_handler, names="value")

def openpose_popup(change):  # Function to display openpose settings if true
  if change["new"]:
    openpose_settings.children = [openpose_toggle, openpose_dropdown, openpose_strength_slider]
  else:
    openpose_settings.children = [openpose_toggle]

def openpose_dropdown_handler(change): # Function to attach the canny dropdown to the controlnet dropdown handler
  controlnet_dropdown_handler("openpose", change["new"])

def openpose_upload_handler(change): # Function to load the path of the uploaded image to the image link
  print(openpose_upload.value)
  if not os.path.exists("/content/openpose/"):
    os.mkdir("/content/openpose/")
  for file_info in openpose_upload.value.items():
    openpose_uploaded_image = file_info[1]["content"]
    with open("/content/openpose/temp.png", "wb") as up:
      up.write(openpose_uploaded_image)
  openpose_link_widget.value = "/content/openpose/temp.png"

openpose_upload = widgets.FileUpload(accept="image/*", multiple=False)
openpose_link_widget = widgets.Text(value=cfg[21] if cfg and (not cfg[21].startswith("/content/openpose/") or os.path.exists("/content/openpose/")) else "", description="OpenPose Link", placeholder="Image link")

openpose_dropdown = widgets.Dropdown(options=controlnet_dropdown_choice, value=controlnet_dropdown_choice[2] if openpose_link_widget.value == "" else controlnet_dropdown_choice[3] if openpose_link_widget.value == "controlnet" else controlnet_dropdown_choice[4] if openpose_link_widget.value == "inpaint" else controlnet_dropdown_choice[0], disabled=False, description="Reference Image")
openpose_toggle = widgets.Checkbox(value=cfg[22] if cfg else False, description="Enable OpenPose")
openpose_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[23] if cfg else 0.7, description="OpenPose Strength")
openpose_settings = widgets.VBox([openpose_toggle])

openpose_popup({"new": openpose_toggle.value})
openpose_upload.observe(openpose_upload_handler, names="value")

canny_dropdown_handler({"new": canny_dropdown.value})
depthmap_dropdown_handler({"new": depthmap_dropdown.value})
openpose_dropdown_handler({"new": openpose_dropdown.value})

canny_dropdown.observe(canny_dropdown_handler, names="value")
depthmap_dropdown.observe(depthmap_dropdown_handler, names="value")
openpose_dropdown.observe(openpose_dropdown_handler, names="value")

canny_toggle.observe(canny_popup, names="value")
depth_map_toggle.observe(depthmap_popup, names="value")
openpose_toggle.observe(openpose_popup, names="value")

controlnet_selections = widgets.VBox([widgets.HTML(value="<hr>"), canny_settings, widgets.HTML(value="<hr>"), depth_settings, widgets.HTML(value="<hr>"), openpose_settings, widgets.HTML(value="<hr>")])
controlnet_settings = widgets.VBox([
    prompts_section,
    model_input_section,
    image_resolution_section,
    generation_parameter_section,
    controlnet_selections,
    scheduler_settings,
    vae_section,
    freeze_widget,
    token_widget,
    widgets.HTML(value="For safety reason, your token <b>won't be saved</b>.")
    ])

# Inpainting Section
inpainting_image_dropdown = widgets.Combobox(
    options=[
        "pre-generated text2image image",
        "pre-generated controlnet image",
        "previous inpainting image"
    ],
    value=cfg[24] if cfg else "pre-generated text2image image",
    description="Inpainting Image",
    ensure_option=False
)
mask_image_widget = widgets.Text(value=cfg[25] if cfg else "", description="Mask Image", placeholder="Image link")
inpainting_toggle = widgets.Checkbox(value=cfg[26] if cfg else False, description="Enable Inpainting")
inpainting_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[27] if cfg else 0.9, description="Inpainting Strength")
inpainting_settings = widgets.VBox([
    inpainting_image_dropdown,
    mask_image_widget,
    inpainting_toggle,
    inpainting_strength_slider,
    widgets.HTML(value="<b>To be updated in the future.</b>")
])

# IP-Adapter Section
def ip_remove_button_on_click(path): # Function to remove images from widget and directory
  os.remove(path)
  ip_grid_button = ip_grid_button_maker(sorted([os.path.join("/content/ip_adapter/", element) for element in os.listdir("/content/ip_adapter/") if os.path.isfile(os.path.join("/content/ip_adapter/", element))]))
  ip_settings.children = [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider, ip_grid_image_html, ip_grid_image, ip_grid_button_html, ip_grid_button]
  if not [os.path.join("/content/ip_adapter/", element) for element in os.listdir("/content/ip_adapter/") if os.path.isfile(os.path.join("/content/ip_adapter/", element))]:
    ip_adapter_dropdown_popup({"new": "refresh_zero"})

def ip_grid_button_maker(list): # Function to make a grid of images and buttons
  list_grid = widgets.GridspecLayout(math.ceil(len(list)/5), 5) if list else widgets.HTML(value="No images has been uploaded.")
  buffer = BytesIO()
  loaded_image_for_grid = []
  if list:
    for i in range(math.ceil(len(list)/5)):
      for j in range(5):
        k = (i*5 + j + 1)
        list_grid[i, j] = widgets.Button(description=f"Remove image {k}", button_style='danger', layout=widgets.Layout(height='auto', width='auto')) if k <= len(list) else widgets.Button(description="", layout=widgets.Layout(height='auto', width='auto'))
        path = list[k - 1] if k <= len(list) else ""
        list_grid[i, j].on_click(lambda b, path=path: ip_remove_button_on_click(path)) if k <= len(list) else None
        loaded_image_for_grid.append(load_image(path)) if k <= len(list) else loaded_image_for_grid.append(load_image("https://huggingface.co/IDK-ab0ut/BFIDIW9W29NFJSKAOAOXDOKERJ29W/resolve/main/placeholder.png"))
    ip_image_grid_maker = make_image_grid([element.resize((1024, 1024)) for element in loaded_image_for_grid], rows=math.ceil(len(list)/5), cols=5)
    ip_image_grid_maker.save(buffer, format = "PNG")
    ip_grid_image.value = buffer.getvalue()
  else:
    ip_settings.children = [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider]
  return list_grid

def ip_adapter_dropdown_popup(change): # Function to show or hide the widgets
  if change["new"] != "None" and change["new"] != "refresh_zero":
    if not os.path.exists("/content/ip_adapter/"):
      os.mkdir("/content/ip_adapter/")
    ip_settings.children = [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider] if not os.listdir("/content/ip_adapter/") else [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider, ip_grid_image_html, ip_grid_image, ip_grid_button_html, ip_grid_button]
  elif change["new"] != "None" and change["new"] == "refresh_zero":
    ip_settings.children = [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider]
  else:
    ip_settings.children = [ip_adapter_dropdown]

def ip_adapter_upload_handler(change): # Function to trigger the UI logic when images are uploaded
  global collected_uploaded_ip_image, ip_grid_button
  if not os.path.exists("/content/ip_adapter/"):
    os.mkdir("/content/ip_adapter/")
  for filename, file_info in ip_image_upload.value.items():
    with open(f"/content/ip_adapter/{filename}", "wb") as up:
      up.write(file_info["content"])
    collected_uploaded_ip_image += f"/content/ip_adapter/{filename},"
  ip_grid_button = ip_grid_button_maker(sorted([os.path.join("/content/ip_adapter/", element) for element in os.listdir("/content/ip_adapter/") if os.path.isfile(os.path.join("/content/ip_adapter/", element))]))
  ip_settings.children = [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider, ip_grid_image_html, ip_grid_image, ip_grid_button_html, ip_grid_button]

collected_uploaded_ip_image = ""
initial_ip_image = [word for word in re.split(r"\s*,\s*", cfg[29]) if word] if cfg else ""
for i, link in enumerate(initial_ip_image):
  if link.startswith("/content/ip_adapter/") and not os.path.exists("/content/ip_adapter"):
    initial_ip_image.remove(link)

ip_grid_image_html = widgets.HTML(value="Uploaded image(s):")
ip_grid_image = widgets.Image()
ip_grid_button_html = widgets.HTML(value="Remove image(s):")
ip_grid_button = ip_grid_button_maker(sorted([os.path.join("/content/ip_adapter/", element) for element in os.listdir("/content/ip_adapter/") if os.path.isfile(os.path.join("/content/ip_adapter/", element))])) if os.path.exists("/content/ip_adapter/") and os.listdir("/content/ip_adapter/") else widgets.GridspecLayout(1, 5)

ip_image_upload = widgets.FileUpload(accept="image/*", multiple=True)
ip_image_link_widget = widgets.Text(value=",".join(initial_ip_image) if cfg else "", description="IP Image Link", placeholder="Image links separated by commas")
ip_adapter_strength_slider = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=cfg[30] if cfg else 0.8, description="Adapter Strength")

ip_adapter_dropdown = widgets.Dropdown(
    options=[
        "ip-adapter-plus_sdxl_vit-h.bin",
        "ip-adapter-plus-face_sdxl_vit-h.bin",
        "ip-adapter_sdxl_vit-h.bin",
        "None"
    ],
    value=cfg[28] if cfg else "None",
    description="IP-Adapter",
)
ip_settings = widgets.VBox()
ip_settings.children = [ip_adapter_dropdown] if ip_adapter_dropdown.value == "None" else [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider] if ip_adapter_dropdown.value != "None" and not os.listdir("/content/ip_adapter") else [ip_adapter_dropdown, ip_image_link_widget, ip_image_upload, ip_adapter_strength_slider, ip_grid_image_html, ip_grid_image, ip_grid_button_html, ip_grid_button]

ip_image_upload.observe(ip_adapter_upload_handler, names="value")
ip_adapter_dropdown.observe(ip_adapter_dropdown_popup, names="value")

# Miscellaneous (generate button and freeze feature)
def reset_evaluate(result): # Function to set every parameter into the default value
  if result == "yes":
    cfg_reset = param_default()
    every_widgets = all_widgets()
    for i in range(len(every_widgets.children)):
      every_widgets.children[i].value = cfg_reset[i]
  reset_display.children = [reset_button]

def reset_button_click(): # Function to show a warning when the reset parameter button is clicked
  reset_yes_button = widgets.Button(description="Yes", button_style='danger')
  reset_no_button = widgets.Button(description="No")

  reset_display.children = [widgets.HTML(value="Are you sure you want to reset all parameters to default? You still can revert it back after rerunning this cell. LoRA and embeddings won't be reset."), widgets.HBox([reset_yes_button, reset_no_button])]
  reset_yes_button._click_handlers.callbacks.clear()
  reset_no_button._click_handlers.callbacks.clear()

  reset_yes_button.on_click(lambda b: reset_evaluate("yes"))
  reset_no_button.on_click(lambda b: reset_evaluate("no"))

submit_button_widget = widgets.Button(disabled=False, button_style='', description="Generate")
dont_spam = widgets.HTML(value="Please <b>don't spam</b> the generate button!")
keep_generating = widgets.HTML(value="You still can generate even though the cell is complete executing.")
submit_display = widgets.VBox([submit_button_widget, dont_spam, keep_generating], layout=widgets.Layout(width="50%"))
reset_button = widgets.Button(description="Reset parameters to default")
reset_display = widgets.VBox([reset_button])

reset_button.on_click(lambda b: reset_button_click())

loaded_model = ""
loaded_pipeline = ""

# Miscellaneous (preset system)
if not os.path.exists(f"{base_path}/Saved Parameters/"):
  os.mkdir(f"{base_path}/Saved Parameters/")

def save_warning_evaluate(result, name):
  if result == "override":
    lora_urls_widget.value, weight_scale_widget.value = lora_reader()
    ti_urls_widget.value, ti_tokens_widget.value = ti_reader()
    save_params = param_constructor()
    save_param(f"{base_path}/Saved Parameters/{name}.json", save_params)
    load_preset_selection_dropdown.options = list_all_saved_preset()

  save_preset_display.children = [save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")] if result != "override" else [widgets.HTML(value=f"Succesfully saved the current parameters as {name}.json in {base_path}/Saved Parameters folder."), save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")]
  save_preset_button._click_handlers.callbacks.clear()
  save_preset_button.on_click(lambda b: save_preset_on_click(save_preset_name_widget.value))

def save_warning_if_preset_exists(name):
  save_warning_back_button = widgets.Button(description="Back")
  save_warning_back_button._click_handlers.callbacks.clear()

  save_preset_display.children = [widgets.HTML(value=f"<span style='color: orange;'>Warning:</span> {name}.json already exists. Saving the current parameters with the same name will override the original saved parameters. Do you wish to continue?"), save_preset_name_widget, widgets.HBox([save_preset_button, save_warning_back_button])]
  save_warning_back_button.on_click(lambda b: save_warning_evaluate("back", name))

  save_preset_button._click_handlers.callbacks.clear()
  save_preset_button.on_click(lambda b: save_warning_evaluate("override", name))

def save_preset_on_click(name):
  if name and name not in list_all_saved_preset():
    lora_urls_widget.value, weight_scale_widget.value = lora_reader()
    ti_urls_widget.value, ti_tokens_widget.value = ti_reader()
    save_params = param_constructor()
    save_param(f"{base_path}/Saved Parameters/{name}.json", save_params)
    save_preset_display.children = [widgets.HTML(value=f"Succesfully saved the current parameters as {name}.json in {base_path}/Saved Parameters folder."), save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")]
    load_preset_selection_dropdown.options = list_all_saved_preset()
  elif name in list_all_saved_preset():
    save_warning_if_preset_exists(name)
  else:
    save_preset_display.children = [widgets.HTML(value="<span style='color: red;'>Error:</span> Name cannot be empty!"), save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")]
    time.sleep(1.5)
    save_preset_display.children = [save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")]

def list_all_saved_preset():
  list_of_saved_parameters = [word.replace(".json" , "") for word in os.listdir(f"{base_path}/Saved Parameters/") if os.path.isfile(os.path.join(f"{base_path}/Saved Parameters/", word)) and word.endswith(".json")]
  return list_of_saved_parameters

def load_preset_on_click(name):
  preset_cfg = load_param(os.path.join(f"{base_path}/Saved Parameters/", f"{name}.json"))
  every_widgets = all_widgets()
  for i in range(len(every_widgets.children)):
    every_widgets.children[i].value = preset_cfg[i]
  lora_reader_upon_starting()
  ti_reader_upon_starting()

save_preset_button = widgets.Button(description="Save current parameters")
save_preset_name_widget = widgets.Text(description="Name", placeholder="Preset name", value="")
save_preset_display = widgets.VBox([save_preset_name_widget, save_preset_button, widgets.HTML(value="Clicking the save button will save the current parameters you're using as a new preset for later use.")])

load_preset_button = widgets.Button(description="Load this preset")
load_preset_selection_dropdown = widgets.Dropdown(description="Select your saved preset")
load_preset_selection_dropdown.options = list_all_saved_preset()
load_preset_display = widgets.VBox([load_preset_selection_dropdown, load_preset_button, widgets.HTML(value="Clicking the load button will override the current parameters.")])

preset_tab = widgets.Tab()
preset_tab.layout = widgets.Layout(width="99%")
preset_tab.children = [save_preset_display, load_preset_display]
preset_tab.set_title(0, "Save Preset")
preset_tab.set_title(1, "Load Preset")

save_preset_button.on_click(lambda b: save_preset_on_click(save_preset_name_widget.value))
load_preset_button.on_click(lambda b: load_preset_on_click(load_preset_selection_dropdown.value))

preset_tab_vbox = widgets.VBox([preset_tab], layout=widgets.Layout(width="50%", align_items="flex-end"))

# Miscellaneous (history)
def history_quick_reference_controlnet_selector(type, path): # Function to input the image as the reference for ControlNet
  global canny_link_widget, canny_toggle, canny_settings, depth_map_link_widget, depth_map_toggle, depth_settings, openpose_link_widget, openpose_toggle, openpose_settings
  if type == "canny":
    canny_link_widget.value = path
    canny_dropdown.value = "Link"
    canny_toggle.value = True
    ui.selected_index = 2
    canny_settings.children = [canny_toggle, canny_dropdown, canny_link_widget, canny_min_slider, canny_max_slider, canny_strength_slider]
  elif type == "depthmap":
    depth_map_link_widget.value = path
    depthmap_dropdown.value = "Link"
    depth_map_toggle.value = True
    ui.selected_index = 2
    depth_settings.children = [depth_map_toggle, depthmap_dropdown, depth_map_link_widget, depth_strength_slider]
  elif type == "openpose":
    openpose_link_widget.value = path
    openpose_dropdown.value = "Link"
    openpose_toggle.value = True
    ui.selected_index = 2
    openpose_settings.children = [openpose_toggle, openpose_dropdown, openpose_link_widget, openpose_strength_slider]
  history_button_handler(path)

def history_quick_reference_second(type, path): # Function to input the image as the reference image
  global ui, ip_adapter_dropdown, reference_image_link_widget, inpainting_image_dropdown, inpainting_toggle
  if type == "img2img":
    reference_image_link_widget.value = path
    history_button_handler(path)
    ui.selected_index = 1
  elif type == "inpainting":
    inpainting_image_dropdown.value = path
    history_button_handler(path)
    inpainting_toggle.value = True
    ui.selected_index = 3
  elif type == "ip":
    if ip_adapter_dropdown.value == "None":
      ip_adapter_dropdown.value = "ip-adapter_sdxl_vit-h.bin"
    if ip_image_link_widget.value == "":
      ip_image_link_widget.value = path
    else:
      ip_image_link_widget.value += "," + path
    history_button_handler(path)
    ui.selected_index = 6
  elif type == "controlnet":
    history_back_button_second = widgets.Button(description="Back", button_style='danger')
    history_image_display_first.children = [widgets.HTML(value="Image will show up here. (from the newest to the oldest)"), history_image_widget, history_image_modification_date, widgets.HBox([history_quick_reference_canny, history_quick_reference_depthmap, history_quick_reference_openpose]), history_back_button_second]

    history_back_button_second._click_handlers.callbacks.clear()
    history_quick_reference_canny._click_handlers.callbacks.clear()
    history_quick_reference_depthmap._click_handlers.callbacks.clear()
    history_quick_reference_openpose._click_handlers.callbacks.clear()

    history_quick_reference_canny.on_click(lambda b: history_quick_reference_controlnet_selector("canny", path))
    history_quick_reference_depthmap.on_click(lambda b: history_quick_reference_controlnet_selector("depthmap", path))
    history_quick_reference_openpose.on_click(lambda b: history_quick_reference_controlnet_selector("openpose", path))
    history_back_button_second.on_click(lambda b: history_quick_reference_first(path))

def history_quick_reference_first(path): # Function to use an image from history to be the reference image of Img2Img, ControlNet, or Inpainting
  history_back_button_first = widgets.Button(description="Back", button_style='danger')
  history_image_display_first.children = [widgets.HTML(value="Image will show up here. (from the newest to the oldest)"), history_image_widget, history_image_modification_date, widgets.HBox([history_quick_reference_img2img, history_quick_reference_controlnet, history_quick_reference_inpainting, history_quick_reference_ip_adapter]), history_back_button_first]

  history_back_button_first._click_handlers.callbacks.clear()
  history_quick_reference_img2img._click_handlers.callbacks.clear()
  history_quick_reference_controlnet._click_handlers.callbacks.clear()
  history_quick_reference_inpainting._click_handlers.callbacks.clear()
  history_quick_reference_ip_adapter._click_handlers.callbacks.clear()

  history_back_button_first.on_click(lambda b: history_button_handler(path))
  history_quick_reference_img2img.on_click(lambda b: history_quick_reference_second("img2img", path))
  history_quick_reference_controlnet.on_click(lambda b: history_quick_reference_second("controlnet", path))
  history_quick_reference_inpainting.on_click(lambda b: history_quick_reference_second("inpainting", path))
  history_quick_reference_ip_adapter.on_click(lambda b: history_quick_reference_second("ip", path))

def history_button_handler(path): # Function to show and replace image from history upon clicking a button
  history_image_widget.value = open(path, "rb").read()
  history_image_modification_date.value = f"Last modification time: {time.strftime('%B, %d %Y %H:%M:%S', time.localtime(os.path.getmtime(path)))}"
  history_image_display_first.children = [widgets.HTML(value="Image will show up here. (from the newest to the oldest)"), history_image_widget, history_image_modification_date, history_quick_reference_button]

  history_quick_reference_button._click_handlers.callbacks.clear()
  history_quick_reference_button.on_click(lambda b: history_quick_reference_first(path))

def grid(list): # Function to make a grid of buttons
  list_grid = widgets.GridspecLayout(math.ceil(len(list)/10), 10) if list else widgets.HTML(value="Nothing in here currently.")
  if list:
    for i in range(math.ceil(len(list)/10)):
      for j in range(10):
        k = (i*10 + j + 1)
        list_grid[i, j] = widgets.Button(description=str(k), layout=widgets.Layout(height='auto', width='auto')) if k <= len(list) else widgets.Button(description="", layout=widgets.Layout(height='auto', width='auto'))
        path = list[k - 1] if k <= len(list) else ""
        list_grid[i, j].on_click(lambda b, path=path: history_button_handler(path)) if k <= len(list) else None
  return list_grid

def history_display(): # Main logic for history
  text2img_listdir = sorted([os.path.join(f"{base_path}/Text2Img", element) for element in os.listdir(f"{base_path}/Text2Img") if element.endswith(".png") and os.path.isfile(os.path.join(f"{base_path}/Text2Img", element))], key=os.path.getmtime, reverse=True) if os.path.exists(f"{base_path}/Text2Img") else []
  controlnet_listdir = sorted([os.path.join(f"{base_path}/ControlNet", element) for element in os.listdir(f"{base_path}/ControlNet") if element.endswith(".png") and os.path.isfile(os.path.join(f"{base_path}/ControlNet", element))], key=os.path.getmtime, reverse=True) if os.path.exists(f"{base_path}/ControlNet") else []
  inpainting_listdir = sorted([os.path.join(f"{base_path}/Inpainting", element) for element in os.listdir(f"{base_path}/Inpainting") if element.endswith(".png") and os.path.isfile(os.path.join(f"{base_path}/Inpainting", element))], key=os.path.getmtime, reverse=True) if os.path.exists(f"{base_path}/Inpainting") else []
  img2img_listdir = sorted([os.path.join(f"{base_path}/Img2Img", element) for element in os.listdir(f"{base_path}/Img2Img") if element.endswith(".png") and os.path.isfile(os.path.join(f"{base_path}/Img2Img", element))], key=os.path.getmtime, reverse=True) if os.path.exists(f"{base_path}/Img2Img") else []

  text2img_list = grid(text2img_listdir)
  controlnet_list = grid(controlnet_listdir)
  inpainting_list = grid(inpainting_listdir)
  img2img_list = grid(img2img_listdir)

  history_accordion = widgets.Accordion(continuous_update = True)
  history_image_modification_date = widgets.HTML()
  history_image_widget = widgets.Image()

  history_image_display_first = widgets.VBox([widgets.HTML(value="Image will show up here. (from the newest to the oldest)"), history_image_widget, history_image_modification_date])
  history_accordion.children = [text2img_list, img2img_list, controlnet_list, inpainting_list]

  history_accordion.set_title(0, "Text-to-Image History")
  history_accordion.set_title(1, "Image-to-Image History")
  history_accordion.set_title(2, "ControlNet History")
  history_accordion.set_title(3, "Inpainting History")
  history_display_vbox = widgets.VBox([history_accordion, history_image_display_first])
  return text2img_list, controlnet_list, inpainting_list, img2img_list

history_quick_reference_button = widgets.Button(description="Use as a reference")
history_quick_reference_img2img = widgets.Button(description="Image-to-image")
history_quick_reference_controlnet = widgets.Button(description="ControlNet")
history_quick_reference_inpainting = widgets.Button(description="Inpainting")
history_quick_reference_ip_adapter = widgets.Button(description="IP-Adapter")

history_quick_reference_canny = widgets.Button(description="Canny")
history_quick_reference_depthmap = widgets.Button(description="DepthMap")
history_quick_reference_openpose = widgets.Button(description="OpenPose")

history_accordion = widgets.Accordion(continuous_update = True)
history_image_modification_date = widgets.HTML()
history_image_widget = widgets.Image()

history_image_display_first = widgets.VBox([widgets.HTML(value="Image will show up here. (from the newest to the oldest)"), history_image_widget, history_image_modification_date], continuous_update = True)
text2img_list, controlnet_list, inpainting_list, img2img_list = history_display()
history_accordion.children = [text2img_list, controlnet_list, inpainting_list, img2img_list]

history_accordion.set_title(0, "Text-to-Image History 🔮✍")
history_accordion.set_title(1, "Image-to-Image History 🔮🎨")
history_accordion.set_title(2, "ControlNet History 🔮🔧")
history_accordion.set_title(3, "Inpainting History 🔮🖌️")

history_display_vbox = widgets.VBox([history_accordion, history_image_display_first], continuous_update = True)

# Accordion, Tab, and  UI display grouping
def checking_the_selected_tab_index(change):
  global tab_selected_index
  tab_selected_index = change["new"]
  if tab_selected_index > 3:
    submit_display.layout.visibility = "hidden"
  else:
    submit_display.layout.visibility = "visible"

ui = widgets.Tab()
ui.children = [widgets.VBox([text2img_settings, reset_display]), widgets.VBox([img2img_settings, reset_display]), widgets.VBox([controlnet_settings, reset_display]), widgets.VBox([inpainting_settings, reset_display]), lora_settings, ti_tab, ip_settings, history_display_vbox]
ui_titles = ["Text-to-image ✍", "Image-to-image 🎨", "ControlNet 🖼️🔧", "Inpainting 🖼️🖌️", "LoRA Settings 📁🖌️", "Textual Inversion 📃🖌️", "IP-Adapter Settings 🖼️📝", "History 🔮📜"]
for i, title in enumerate(ui_titles):
  ui.set_title(i, title)

ui.observe(checking_the_selected_tab_index, names="selected_index")
checking_the_selected_tab_index({"name": "selected_index", "new": ui.selected_index, "old": None, "type": "change", "owner": ui})
# ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

# The IPyWidgets handler

def widget(): # Function to display the ui and update the history
    text2img_list, controlnet_list, inpainting_list, img2img_list = history_display()
    history_accordion.children = [text2img_list, controlnet_list, inpainting_list, img2img_list]
    history_display_vbox.children = [history_accordion, history_image_display_first]

    clear_output()
    submit_display.layout.visibility = "visible"
    lora_add.layout.display = "inline-block"
    ti_add.layout.display = "inline-block"
    display(ui, widgets.HBox([submit_display, preset_tab_vbox]))

def submit(_): # Function to trigger the main logic
    submit_display.layout.visibility = "hidden"
    lora_add.layout.display = "none"
    ti_add.layout.display = "none"
    lora_urls_widget.value, weight_scale_widget.value = lora_reader()
    ti_urls_widget.value, ti_tokens_widget.value = ti_reader()
    ip_image_link_widget.value += "," + ",".join([word for word in re.split(r"\s*,\s*", collected_uploaded_ip_image) if word not in ip_image_link_widget.value])
    submit_button()

# Main logic
def submit_button():
    torch.backends.cudnn.benchmark=True
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:16"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    Freeze = freeze_widget.value

    # Handling Google Drive and seed
    filename = os.path.join(base_path, "random_number.json")
    saved_number = load_number(filename)
    if not Freeze:
        # Generate a new random number if Freeze is False
        random_number = random.randint(1, 1000000000000)
        save_number(filename, random_number)
        saved_number = load_number(filename)
    else:
        # Use the saved number if Freeze is True
        if saved_number is not None:
            saved_number = saved_number
        else:
            print("No saved seed found. Generating new one...")
            random_number = random.randint(1, 1000000000000)
            save_number(filename, random_number)
            saved_number = load_number(filename)

    # Handling user's input⬇️
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
    Prompt = prompt_widget.value
    Model = model_widget.value
    Model_Format = model_format_widget.value
    Negative_Prompt = negative_prompt_widget.value

    Width = width_slider.value
    Height = height_slider.value
    Steps = steps_slider.value
    Scale = scale_slider.value
    VAE_Link = vae_link_widget.value
    VAE_Config = vae_config.value
    Clip_Skip = clip_skip_slider.value

    Reference_Image = reference_image_link_widget.value
    Denoising_Strength = denoising_strength_slider.value

    Scheduler = scheduler_dropdown.value
    Karras = karras_bool.value
    V_Prediction = vpred_bool.value
    SGMUniform = sgmuniform_bool.value
    Rescale_betas_to_zero_SNR = res_betas_zero_snr.value

    LoRA_URLs = lora_urls_widget.value
    Weight_Scale = weight_scale_widget.value
    Token = token_widget.value

    Textual_Inversion_URLs = ti_urls_widget.value
    Textual_Inversion_Tokens = ti_tokens_widget.value

    minimum_canny_threshold = canny_min_slider.value
    maximum_canny_threshold = canny_max_slider.value
    Canny_Link = canny_link_widget.value
    Canny = canny_toggle.value
    Canny_Strength = canny_strength_slider.value

    DepthMap_Link = depth_map_link_widget.value
    Depth_Map = depth_map_toggle.value
    Depth_Strength = depth_strength_slider.value

    OpenPose_Link = openpose_link_widget.value
    Open_Pose = openpose_toggle.value
    Open_Pose_Strength = openpose_strength_slider.value

    Inpainting_Image = inpainting_image_dropdown.value
    Mask_Image = mask_image_widget.value
    Inpainting = inpainting_toggle.value
    Inpainting_Strength = inpainting_strength_slider.value

    IP_Adapter = ip_adapter_dropdown.value
    IP_Image_Link = ip_image_link_widget.value
    IP_Adapter_Strength = ip_adapter_strength_slider.value
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

    # Selecting image
    last_generation_loading = os.path.join(base_path, "last_generation.json")
    global pipeline_type, tab_selected_index
    selected_tab_for_pipeline = tab_selected_index

    if Canny and selected_tab_for_pipeline == 2:
        if Canny_Link == "inpaint":
            Canny_link = load_last(last_generation_loading, 'inpaint')
        elif Canny_Link == "controlnet":
            Canny_link = load_last(last_generation_loading, 'controlnet')
        elif not Canny_Link:
            Canny_link = load_last(last_generation_loading, 'text2img')
        else:
            Canny_link = Canny_Link
        if Canny_link or os.path.exists(Canny_link):
            pipeline_type = "controlnet"
    else:
        Canny_link = ""

    if Depth_Map and selected_tab_for_pipeline == 2:
        if DepthMap_Link == "inpaint":
            Depthmap_Link = load_last(last_generation_loading, 'inpaint')
        elif DepthMap_Link == "controlnet":
            Depthmap_Link = load_last(last_generation_loading, 'controlnet')
        elif not DepthMap_Link:
            Depthmap_Link = load_last(last_generation_loading, 'text2img')
        else:
            Depthmap_Link = DepthMap_Link
        if Depthmap_Link or os.path.exists(Depthmap_Link):
            pipeline_type = "controlnet"
    else:
        Depthmap_Link = ""

    if Open_Pose and selected_tab_for_pipeline == 2:
        if OpenPose_Link == "inpaint":
            Openpose_Link = load_last(last_generation_loading, 'inpaint')
        elif OpenPose_Link == "controlnet":
            Openpose_Link = load_last(last_generation_loading, 'controlnet')
        elif not OpenPose_Link:
            Openpose_Link = load_last(last_generation_loading, 'text2img')
        else:
            Openpose_Link = OpenPose_Link
        if Openpose_Link or os.path.exists(Openpose_Link):
            pipeline_type = "controlnet"
    else:
        Openpose_Link = ""

    active_inpaint = False
    if Inpainting and selected_tab_for_pipeline == 3:
        if Canny or Depth_Map or Open_Pose:
            raise TypeError("You checked both ControlNet and Inpainting, which will cause incompatibility issues during your run. As of now, there's no alternative way to merge StableDiffusionXLControlNetPipeline and StableDiffusionXLInpaintingPipeline without causing any issues. Perhaps you want to use only one of them?")
        if not Mask_Image:
            raise ValueError("You checked Inpainting while you're leaving Mask_Image empty. Mask_Image is required for Inpainting!")
        if Inpainting_Image == "pre-generated text2image image":
            inpaint_img = load_last(last_generation_loading, 'text2img')
        elif Inpainting_Image == "pre-generated controlnet image":
            inpaint_img = load_last(last_generation_loading, 'controlnet')
        elif Inpainting_Image == "previous inpainting image":
            inpaint_img = load_last(last_generation_loading, 'inpaint')
        else:
            inpaint_image = Inpainting_Image
        if inpaint_img is not None and os.path.exists(inpaint_img):
            pipeline_type = "inpaint"
            inpaint_image = load_image(inpaint_img).resize((1024, 1024))
            mask_image = load_image(Mask_Image).resize((1024, 1024))
            active_inpaint = True
            display(make_image_grid([inpaint_image, mask_image], rows=1, cols=2))

    if Reference_Image and selected_tab_for_pipeline == 1:
      ref_image = load_image(Reference_Image)
      if ref_image or os.path.exists(ref_image):
        pipeline_type = "img2img"
    else:
      ref_image = ""

    if not IP_Image_Link and IP_Adapter != "None":
        raise ValueError(f"You selected {IP_Adapter}, but left the IP_Image_Link empty. Please change the IP_Adapter to None or add at least one image in IP_Image_Link!")
    if selected_tab_for_pipeline == 0 or (not Canny_link and not Depthmap_Link and not Openpose_Link and not active_inpaint and not ref_image):
        pipeline_type = "text2img"

    # Saving parameters config 1st phase
    params = param_constructor()
    save_param(f"{base_path}/Saved Parameters/main_parameters.json", params)
    if os.path.exists(os.path.join(f"{base_path}", "parameters.json")):
      os.remove(os.path.join(f"{base_path}", "parameters.json"))

    # Checking if previous loaded model or pipeline is the same as the new one
    global loaded_model, loaded_pipeline
    if loaded_model and loaded_model != model_widget.value:
        restart(model_widget.value, loaded_model)
    if loaded_pipeline and loaded_pipeline != pipeline_type:
        restart(pipeline_type, loaded_pipeline)

    # Logic to handle ControlNet and/or MultiControlNets
    if Canny and Canny_link is not None:
        if "canny" not in loaded_controlnet_model:
          global canny_model
          canny_model = ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True, low_cpu_mem_usage=True)
          loaded_controlnet_model[0] = "canny"
          controlnets[0] = canny_model
        print("🏞️ | Converting image with Canny Edge Detection...")
        c_img = load_image(Canny_link)
        image_canny = np.array(c_img)
        image_canny = cv2.Canny(image_canny, minimum_canny_threshold, maximum_canny_threshold)
        image_canny = image_canny[:, :, None]
        image_canny = np.concatenate([image_canny, image_canny, image_canny], axis=2)
        canny_image = ImagePIL.fromarray(image_canny)
        print("✅ | Canny Edge Detection is complete.")
        time.sleep(1)
        display(make_image_grid([c_img, canny_image.resize((1024, 1024))], rows=1, cols=2))
        images[0] = canny_image.resize((1024, 1024))
        controlnets_scale[0] = Canny_Strength

    if Depth_Map and Depthmap_Link is not None:
        if "depth" not in loaded_controlnet_model:
          global depthmap_model
          loaded_controlnet_model[1] = "depth"
          depthmap_model = ControlNetModel.from_pretrained("diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True, low_cpu_mem_usage=True).to("cuda")
          controlnets[1] = depthmap_model
        print("🏞️ | Converting image with Depth Map...")
        image_depth = load_image(Depthmap_Link).resize((1024, 1024))
        depth_estimator = pipe("depth-estimation")
        depth_map = get_depth_map(image_depth, depth_estimator).unsqueeze(0).half().to("cpu")
        images[1] = depth_map
        depth_map_display = ImagePIL.fromarray(get_depth_map_display(image_depth, depth_estimator))
        print("✅ | Depth Map is complete.")
        controlnets_scale[1] = Depth_Strength
        time.sleep(1)
        display(make_image_grid([image_depth, depth_map_display], rows=1, cols=2))

    if Open_Pose and Openpose_Link is not None:
        if "openpose" not in loaded_controlnet_model:
          global openpose, openpose_model
          loaded_controlnet_model[2] = "openpose"
          openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet").to("cpu")
          openpose_model = ControlNetModel.from_pretrained("thibaud/controlnet-openpose-sdxl-1.0", torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda")
          controlnets[2] = openpose_model
        print("🏞️ | Converting image with Open Pose...")
        image_openpose = load_image(Openpose_Link)
        openpose_image = openpose(image_openpose)
        images[2] = openpose_image.resize((1024, 1024))
        print("✅ | Open Pose is done.")
        controlnets_scale[2] = Open_Pose_Strength
        display(make_image_grid([image_openpose, openpose_image.resize((1024, 1024))], rows=1, cols=2))

    # Logic to handle VAE
    if VAE_Link and VAE_Config:
        if not os.path.exists("/content/VAE"):
            os.mkdir("VAE")
        vae_filename = VAE_Link.replace("/", "_").replace(".", "_") + ".safetensors"
        if VAE_Link.startswith("http"):
            if "civitai.com" in VAE_Link:
                if "?" in VAE_Link or "&" in VAE_Link:
                    vae_link = VAE_Link + "&token=" + Token
                else:
                    vae_link = VAE_Link + "token=" + Token
            else:
                vae_link = VAE_Link
            if not os.path.exists(f"/content/VAE/{vae_filename}"):
                !cd /content/VAE; wget -O "$vae_filename" "$vae_link"
                !cd /content/VAE; wget -O config.json "$VAE_Config"
        vae_path = f"/content/VAE/{vae_filename}" if VAE_Link.startswith("http") else VAE_Link
        vae = AutoencoderKL.from_single_file(vae_path, config="/content/VAE/config.json", torch_dtype=torch.float16, local_files_only=True)
    elif VAE_Link and not VAE_Config:
        vae = None
        print("You inputted a VAE link, but not the config. Config is essential to load the model.")
        print("Skipping VAE...")

    # Logic to differentiate if the model is Hugging Face's repository
    global pipeline
    if Model.count("/") == 1 and (not Model.startswith("https://") or not Model.startswith("http://")):
        if all(cn is None for cn in controlnets) and pipeline_type != "img2img" and not active_inpaint and (pipeline_type != "text2img" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
            if VAE_Link:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, torch_dtype=torch.float16).to("cuda")
        elif active_inpaint and pipeline_type != "img2img" and all(cn is None for cn in controlnets) and (pipeline_type != "inpaint" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
            if VAE_Link:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, torch_dtype=torch.float16).to("cuda")
        elif pipeline_type != "img2img" and (pipeline_type != "controlnet" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
            if VAE_Link:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, controlnet=[element for element in controlnets if element], vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, controlnet=[element for element in controlnets if element], torch_dtype=torch.float16).to("cuda")
        elif pipeline_type == "img2img" and (pipeline_type != "img2img" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
            if VAE_Link:
                pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(Model, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLImg2ImgPipeline.from_pretrained(Model, torch_dtype=torch.float16).to("cuda")
    else:
        if not os.path.exists("/content/Checkpoint"):
            os.mkdir("Checkpoint")
        if ".ckpt" in Model_Format:
            format = ".ckpt"
        elif ".safetensors" in Model_Format:
            format = ".safetensors"
        checkpoint_name = f"checkpoint_model{format}"
        if Token and "civitai.com" in Model:
            if "?" in Model or "&" in Model:
                checkpoint_link = f"{Model}&token={Token}"
            else:
                checkpoint_link = f"{Model}token={Token}"
        else:
            checkpoint_link = Model
        Model_folder = Model.replace("/", "_").replace(".", "_")
        Model_path = f"/content/Checkpoint/{Model_folder}/{checkpoint_name}"
        Model_path_folder = f"/content/Checkpoint/{Model_folder}"
        if not os.path.exists(Model_path):
            if not os.path.exists(Model_path_folder):
                os.mkdir(Model_path_folder)
            !cd "$Model_path_folder"; wget -O "$checkpoint_name" "$checkpoint_link"
        try:
            if all(cn is None for cn in controlnets) and pipeline_type != "img2img" and not active_inpaint and (pipeline_type != "text2img" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
                if VAE_Link:
                    pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, vae=vae, torch_dtype=torch.float16, variant="fp16").to("cuda")
                else:
                    pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, torch_dtype=torch.float16, variant="fp16").to("cuda")
            elif pipeline_type != "img2img" and active_inpaint and all(cn is None for cn in controlnets) and (pipeline_type != "inpaint" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
                if VAE_Link:
                    pipeline = AutoPipelineForInpainting.from_single_file(Model_path, vae=vae, torch_dtype=torch.float16, variant="fp16").to("cuda")
                else:
                    pipeline = AutoPipelineForInpainting.from_single_file(Model_path, torch_dtype=torch.float16, variant="fp16").to("cuda")
            elif pipeline_type != "img2img" and (pipeline_type != "controlnet" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
                if VAE_Link:
                    pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, controlnet=[element for element in controlnets if element], vae=vae, torch_dtype=torch.float16, variant="fp16").to("cuda")
                else:
                    pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, controlnet=[element for element in controlnets if element], torch_dtype=torch.float16, variant="fp16").to("cuda")
            elif pipeline_type == "img2img" and (pipeline_type != "img2img" or not loaded_pipeline) and (not loaded_model or model_widget.value != loaded_model):
                if VAE_Link:
                    pipeline = StableDiffusionXLImg2ImgPipeline.from_single_file(Model_path, vae=vae, torch_dtype=torch.float16, variant="fp16").to("cuda")
                else:
                    pipeline = StableDiffusionXLImg2ImgPipeline.from_single_file(Model_path, torch_dtype=torch.float16, variant="fp16").to("cuda")
        except (ValueError, OSError):
            pass
            os.remove(Model_path)
            if not Token and "civitai.com" in Model:
                Warning = "You inputted a CivitAI's link, but your token is empty. It's possible that you got unauthorized access during the download."
            else:
                Warning = "Did you input the correct link? Or did you use the correct format?"
            raise TypeError(f"The link ({Model}) contains unsupported file or the download was corrupted. {Warning}")

    if IP_Adapter != "None":
        pipeline.image_encoder = CLIPVisionModelWithProjection.from_pretrained(
            "h94/IP-Adapter",
            subfolder="models/image_encoder",
            torch_dtype=torch.float16,
        ).to("cuda")

    if not loaded_model:
        loaded_model = model_widget.value
    if not loaded_pipeline:
        loaded_pipeline = pipeline_type

    # Seed, safety checker, and memory attention (Xformers)
    pipeline.enable_xformers_memory_efficient_attention()
    generator = torch.Generator("cpu").manual_seed(saved_number)
    pipeline.safety_checker = None

    # Handling schedulers
    Prediction_type = "v_prediction" if V_Prediction else "epsilon"
    scheduler_args = {"prediction_type": Prediction_type,
                           "use_karras_sigmas": Karras,
                           "rescale_betas_zero_snr": Rescale_betas_to_zero_SNR
                           }
    if SGMUniform:
      scheduler_args["timestep_spacing"] = "trailing"
    Scheduler_used = ["", f"{Scheduler} ", "", "", ""]
    Scheduler_used[0] = "V-Prediction " if Prediction_type == "v_prediction" else ""
    Scheduler_used[2] = "Karras " if Karras else ""
    Scheduler_used[3] = "SGMUniform " if SGMUniform else ""
    Scheduler_used[4] = "with zero SNR betas rescaling" if Rescale_betas_to_zero_SNR else ""
    if Scheduler == "DPM++ 2M":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM++ 2M SDE":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, algorithm_type="sde-dpmsolver++", **scheduler_args)
    elif Scheduler == "DPM++ SDE":
        pipeline.scheduler = DPMSolverSinglestepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM2":
        pipeline.scheduler = KDPM2DiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DPM2 a":
        pipeline.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DDPM":
        pipeline.scheduler = DDPMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Euler":
        pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Euler a":
        pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "Heun":
        pipeline.scheduler = HeunDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "LMS":
        pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DEIS":
        pipeline.scheduler = DEISMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "UniPC":
        pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)
    elif Scheduler == "PNDM":
        pipeline.scheduler = PNDMScheduler.from_config(pipeline.scheduler.config, **scheduler_args)

    # Prompt weighting using Compel
    compel = Compel(tokenizer=[pipeline.tokenizer, pipeline.tokenizer_2], text_encoder=[pipeline.text_encoder, pipeline.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True], truncate_long_prompts=False)
    conditioning, pooled = compel([Prompt, Negative_Prompt])

    # Logic to load LoRA(s)
    if LoRA_URLs:
        # Preprocessing the urls and weight before downloading
        lora_list = []
        lora_path = []
        unique_lora_urls = []
        lora_links = [word for word in re.split(r"\s*,\s*", LoRA_URLs) if word]
        if not os.path.exists("/content/LoRAs"):
            os.mkdir("LoRAs")
        if not Weight_Scale:
            scales_string = ["1"] * len(lora_links)
        elif Weight_Scale and len(re.split(r",| ,", Weight_Scale)) < len(lora_links):
            scales_string = re.split(r",| ,", Weight_Scale)
            for j in range(len(lora_links) - len(scales_string)):
                scales_string.append("1")
        else:
            scales_string = re.split(r"\s*,\s*", Weight_Scale)
        scales = [float(num) for num in scales_string if num]

        # Downloading the files and removing any duplicates
        for i, link in enumerate(lora_links, start=1):
          if link not in unique_lora_urls:
            unique_lora_urls.append(link)
            path_file = os.path.join("/content/LoRAs", link.replace("/", "_").replace(".", "_"))
            if not os.path.exists(path_file) and "http" in link:
                os.makedirs(path_file)
            lora_name = link.replace("/", "_").replace(".", "_")
            lora_file_name = f"lora_{lora_name}.safetensors"
            if "civitai.com" in link and Token:
                if "&" in link or "?" in link:
                    civit_link = f"{link}&token={Token}"
                else:
                    civit_link = f"{link}?token={Token}"
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$civit_link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            elif not link.startswith("/content"):
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            else:
                if link.startswith("/content/gdrive/MyDrive"):
                    constructed_gdrive_link = link
                else:
                    constructed_gdrive_link = f"/content/gdrive/MyDrive/{link}"
                link_from_gdrive = constructed_gdrive_link.split("/")
                lora_path.append("/".join([word for word in link_from_gdrive if ".safetensors" not in word]))
                lora_list.append(link_from_gdrive[-1])

        # Loading the downloaded weights into the model
        lora_weights = [word for word in lora_list if word.endswith(".safetensors")]
        lora_names = [word.replace(".safetensors", "") for word in lora_weights]
        for p in range(len(lora_weights)):
            try:
                pipeline.load_lora_weights(f"{lora_path[p]}/{lora_weights[p]}", adapter_name=lora_names[p])
            except Exception as e:
                print(f"Skipping {lora_weights[p]}. Reason: {e}")
        pipeline.set_adapters(lora_names, adapter_weights=scales)
    torch.cuda.empty_cache()

    # Logic to handle the embeddings or textual inversion
    if Textual_Inversion_URLs:
        # Preprocessing the urls and weight before downloading
        ti_list = []
        ti_path = []
        unique_ti_urls = []
        ti_links = [word for word in re.split(r"\s*,\s*", Textual_Inversion_URLs)]
        ti_tokens = [word for word in re.split(r"\s*\s*", Textual_Inversion_Tokens)]
        if not os.path.exists("/content/Embeddings"):
            os.mkdir("/content/Embeddings")

        # Downloading the files and removing any duplicates
        for i, link in enumerate(ti_links, start=1):
          if link not in unique_ti_urls and link:
            unique_ti_urls.append(link)
            ti_path_file = os.path.join("/content/LoRAs", link.replace("/", "_").replace(".", "_"))
            if not os.path.exists(ti_path_file) and "http" in link:
                os.makedirs(ti_path_file)
            ti_name = link.replace("/", "_").replace(".", "_")
            ti_file_name = f"embeddings_{ti_name}.safetensors"
            if "civitai.com" in link and Token:
                if "&" in link or "?" in link:
                    ti_civit_link = f"{link}&token={Token}"
                else:
                    ti_civit_link = f"{link}?token={Token}"
                if not os.path.isfile(os.path.join(ti_path_file, ti_file_name)):
                    !cd "$ti_path_file"; wget -O "$ti_file_name" "$ti_civit_link"
                ti_list.append(ti_file_name)
                ti_path.append(ti_path_file)
            elif not link.startswith("/content"):
                if not os.path.isfile(os.path.join(ti_path_file, ti_file_name)):
                    !cd "$ti_path_file"; wget -O "$ti_file_name" "$link"
                ti_list.append(ti_file_name)
                ti_path.append(ti_path_file)
            else:
                if link.startswith("/content/gdrive/MyDrive"):
                    ti_constructed_gdrive_link = link
                else:
                    ti_constructed_gdrive_link = f"/content/gdrive/MyDrive/{link}"
                ti_link_from_gdrive = ti_constructed_gdrive_link.split("/")
                ti_path.append("/".join([word for word in ti_link_from_gdrive if ".safetensors" not in word]))
                ti_list.append(ti_link_from_gdrive[-1])
          else:
            ti_tokens.pop(i)

        # Loading the downloaded weights into the model
        for p in range(len(ti_list)):
            try:
                ti_dict = load_file(f"{ti_path[p]}/{ti_list[p]}")
                pipeline.load_textual_inversion(ti_dict["clip_g"], token=ti_tokens[p], text_encoder=pipeline.text_encoder_2, tokenizer=pipeline.tokenizer_2)
                pipeline.load_textual_inversion(ti_dict["clip_l"], token=ti_tokens[p], text_encoder=pipeline.text_encoder, tokenizer=pipeline.tokenizer)
            except Exception as e:
                print(f"Skipping {ti_list[p]}. Reason: {e}")
    torch.cuda.empty_cache()

    # Logic to handle image(s) for IP-Adapter + display
    if IP_Adapter != "None":
        # Loading the images
        adapter_image = []
        simple_Url = [word for word in re.split(r"\s*,\s*", IP_Image_Link) if word]
        for link in simple_Url:
            adapter_image.append(load_image(link))

        # Creating the display
        adapter_display = [element for element in adapter_image]
        if len(adapter_image) % 3 == 0:
            row = int(len(adapter_image)/3)
        else:
            row = int(len(adapter_image)/3) + 1
            for i in range(3*row - len(adapter_image)):
                adapter_display.append(load_image("https://huggingface.co/IDK-ab0ut/BFIDIW9W29NFJSKAOAOXDOKERJ29W/resolve/main/placeholder.png"))
        print("Image(s) for IP-Adapter:")
        display(make_image_grid([element.resize((1024, 1024)) for element in adapter_display], rows=row, cols=3))

        # Loading the images to the IP-Adapter
        image_embeds = [adapter_image]
        pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name=IP_Adapter, low_cpu_mem_usage=True)
        pipeline.set_ip_adapter_scale(IP_Adapter_Strength)
    torch.cuda.empty_cache()

    # Generate
    if all(cn is None for cn in controlnets) and not active_inpaint and pipeline_type != "img2img": # For Text2Img
        image_save = "[Text-to-Image]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator
            ).images[0]
            pipeline.unload_ip_adapter()
    elif pipeline_type != "img2img" and active_inpaint and all(cn is None for cn in controlnets): # For Inpainting
        image_save = "[Inpainting]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                image=inpaint_image,
                mask_image=mask_image,
                generator=generator,
                strength=Inpainting_Strength
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
                image=inpaint_image,
                mask_image=mask_image,
                strength=Inpainting_Strength
            ).images[0]
            pipeline.unload_ip_adapter()
    elif pipeline_type != "img2img": # For ControlNet
        image_save = "[ControlNet]"
        if Inpainting:
          if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                clip_skip=Clip_Skip,
                num_inference_steps=Steps,
                generator=generator,
                width=Width,
                height=Height,
                image=[element for element in images if element is not None],
                controlnet_conditioning_scale=[element for element in controlnets_scale if element],
                guidance_scale=Scale
            ).images[0]
          else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
                image=[element for element in images if element is not None],
                controlnet_conditioning_scale=[element for element in controlnets_scale if element]
            ).images[0]
    else: # For Img2img
        image_save = "[Image-to-Image]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                image=ref_image,
                strength=Denoising_Strength,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                image=ref_image,
                strength=Denoising_Strength,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator
            ).images[0]
            pipeline.unload_ip_adapter()

    # Saving the image
    current_time = time.localtime()
    formatted_time = time.strftime("[%H-%M-%S %B %d, %Y]", current_time)
    if Save_and_Connect_To_GDrive:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/gdrive/MyDrive/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/gdrive/MyDrive/ControlNet"
        elif image_save == "[Inpainting]":
            image_save_path = "/content/gdrive/MyDrive/Inpainting"
        else:
            image_save_path = "/content/gdrive/MyDrive/Img2Img"
    else:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/ControlNet"
        elif image_save == "[Inpainting]":
            image_save_path = "/content/Inpainting"
        else:
            image_save_path = "/content/Img2Img"
    if not os.path.exists(image_save_path):
            os.makedirs(image_save_path)
    split_prompt = re.split("\s*,\s*", Prompt.replace("<", "").replace(">", "").replace(":", "_").replace(";", "_"))
    prompt_name = " ".join(split_prompt)
    generated_image_raw_filename = f"{image_save} {formatted_time} {prompt_name}"
    generated_image_filename = generated_image_raw_filename[:251] if len(generated_image_raw_filename) > 255 else generated_image_raw_filename
    generated_image_savefile = f"{image_save_path}/{generated_image_filename}.png"
    image.save(generated_image_savefile)
    widget()

    # Saving parameters config 2nd phase
    params = param_constructor()
    save_param(f"{base_path}/Saved Parameters/main_parameters.json", params)

    # Handling last generated image
    last_generation_json = os.path.join(base_path, "last_generation.json")
    save_last(last_generation_json, generated_image_savefile, image_save)

    # Displaying the image, seed, and scheduler
    display(image)
    print(f"Scheduler: {''.join(Scheduler_used)}")
    print(f"Seed: {saved_number}")
    print(f"Image is saved at {generated_image_savefile}.")
    torch.cuda.empty_cache()

# Lists to prevent ControlNet models from being loaded twice
loaded_controlnet_model = [None] * 3
controlnets = [None] * 3
images = [None] * 3
controlnets_scale = [None] * 3

# Assigning the click event for submit_button_widget, loading lora urls from saved parameter, and displaying the ui
submit_button_widget.on_click(submit)
lora_reader_upon_starting()
ti_reader_upon_starting()
widget()


###<font color="black"> » <b><font color="purple">Information </b>✏️📄</font> <font color="black"> «
#####ㅤ
<small>• Text2Img image is saved in Text2Img folder. </small>

<small>• ControlNet-generated image is saved in ControlNet folder. (requires **Canny**, **Depth Map**, and/or **Open Pose** to be checked, as well as the direct link to the reference image) </small>

<small>• Inpainting-generated image is saved in Inpainting folder. (requires **Inpainting** to be checked, as well as inputting the image and the mask image)</small>

<small> • You can't combine Inpainting and ControlNet.</small>

<small>• IP-Adapter doesn't change the image name.</small>

<small>• You can load LoRAs from your Google Drive by inputting their path. As of now, only LoRAs are supported. </small>

<small>• For ControlNet, leave the image link blank to use the last generated Text2Img image as the reference. Input "inpaint" to use the last generated Inpainting image. And lastly, input "controlnet" to use the last generated ControlNet image. (requires **Canny**, **Depth Map**, **Inpainting**, and/or **Open Pose** to be checked) </small>

***

###<font color="black"> » <b><font color="cyan">Guide </b>🚶🏻📋</font> <font color="black"> «
#####ㅤ

<small> **Prompt:** Basically, this one tells the AI what do you want to see in the image. Sometimes, you have to be strict with your words to align the image with your imagination.

<small> **Model (**checkpoint**):** A saved state during an intense training. This is required to generate the image. The type of model you inputted affects the overall style.

<small> **Model Format:** This is pretty self-explanatory. If you want to use .safetensors model, then set it to "Safe Tensors."

<small> **Steps:** It's simply how many iterations the AI will do in order to generate the image. More doesn't always better. You can look for references online.

<small> **Scale (**Guidance Scale**):** This affects how closely related the image with the prompt. High value can be precise, but low value can add extra uniqueness.

<small> **VAE:** Stands for Variational Autoencoder. It basically controls the color of your image.

<small> **Clip Skip:** Lets the AI skip set amount of layers during generation.

<small> **LoRA:** Stands for Low-Rank Adaptation. It holds weight to be "fed" to the AI. In simple terms, LoRA guides the AI to draw specific characters, style, poses, and so much more. You also need to specify the LoRA's scale, similar to **Scale**.

<small> **ControlNet:** Basically a strict instruction based on the inputted image to generate image closely related to the reference.

<small> **Inpainting:** Redrawing an image, but with certain parts of the image changed, just like editing with Photoshop, but AI does the job for you.

<small> **IP-Adapter:** Similar to LoRA, but only follows the inputted image(s). This is stricter than LoRA and sometimes lacks generalization.

<small> **Negative Prompt:** The reverse version of **Prompt**. Instead of telling the AI what do you want, this tells the AI about what do you want to be removed from the image.